In [100]:
import pandas as pd
import numpy as np
import sklearn
import graphviz
import os

import matplotlib 
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score

from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans
import time
from operator import itemgetter

import xgboost as xgb

from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

import sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [10]:
#path_import = "/Users/ppx/Desktop/"
#import_file = "the_result_file.csv"
path = 'SMSSpamCollection.data'

In [11]:
GNB = GaussianNB()

DTC = tree.DecisionTreeClassifier()

ID3 = tree.DecisionTreeClassifier(criterion="entropy")

decision_stump = GradientBoostingClassifier(n_estimators=100,
                                 learning_rate=1.0,
                                 max_depth=1,
                                 random_state=0)

MLP = MLPClassifier(solver='lbfgs',
                    alpha=1,
                    hidden_layer_sizes=(20, 10))

KNN = KNeighborsClassifier()

BC = BaggingClassifier(n_estimators=50)

ABC = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1.0,
                         algorithm='SAMME.R',
                         random_state=None)

RFC = RandomForestClassifier(n_estimators=10,
                             criterion='gini',
                             max_depth=None,
                             min_samples_split=2,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features='auto',
                             max_leaf_nodes=None,
                             min_impurity_decrease=0.0,
                             min_impurity_split=None,
                             bootstrap=True,
                             oob_score=False,
                             n_jobs=1,
                             random_state=None,
                             verbose=0,
                             warm_start=False,
                             class_weight=None)




max_depth=7
learning_rate=0.05
n_estimators=50
gamma=0.5
reg_alpha=0.1


GBM = xgb.XGBClassifier(max_depth = max_depth, 
                        n_estimators = n_estimators, 
                        learning_rate = learning_rate,
                        gamma = gamma,
                        reg_alpha= reg_alpha)


In [99]:
colnames = ['Accuracy', 'AUC', 'Moyenne CV', 'Std CV', 'Temps']
algos = [GNB, KNN, DTC, ID3, BC, RFC, ABC, decision_stump, GBM]
indexs = ['GNB', 'KNN', 'DTC', 'id3', 'BC', 'RFC', 'ABC', 'DS','GBM']
clfs = dict()
for x in range(len(algos)):
    clfs[indexs[x]] = algos[x]

modes = [None, 'normalized', 'min_max_ed', 'pca']

# Mise en forme donnée

In [102]:
data = pd.read_csv(path, delimiter='	',header=None, na_values='?')

In [103]:
mf = 500
vectorizer = CountVectorizer(max_features=mf)
X = vectorizer.fit_transform(data[1])
analyze = vectorizer.build_analyzer()
X = X.toarray()
X = pd.DataFrame(X)

In [104]:
target = pd.DataFrame(data[0])
target = target.replace('spam', 1)
target = target.replace('ham', 0)


# Qualité Apprentissage

In [86]:
class Qualite:
    def __init__(self, _modele, _data, _y, _y_pred, _y_pred_proba, _cv=10, _print_mode=True):
        
        self.accuracy = sklearn.metrics.accuracy_score(_y, _y_pred)
        self.AUC = sklearn.metrics.roc_auc_score(_y, _y_pred)
        self.cross_validation = cross_val_score(_modele, _data, y=_y, cv=_cv)
        self.moyenne = np.mean(self.cross_validation)
        self.ecart_type = np.std(self.cross_validation)
        self.precision = average_precision_score(_y, _y_pred)
        
        self.roc = ROC_curves(pred=_y_pred_proba, res=_y)
        
        
        if _print_mode:
            self.roc.plot_it()

# ROC

In [87]:
def TP(theta, pred, result):
    count = 0
    i = 0
    for x in (result):
        if pred[i][1] > theta and x == 1:
            count += 1
        i += 1
    return count


def FP(theta, pred, result):
    count = 0
    i = 0
    for x in (result):
        if pred[i][1] >= theta and x == 0:
            count += 1
        i += 1
    return count


def TN(theta, pred, result):
    count = 0
    i = 0
    for x in (result):
        if pred[i][1] < theta and x == 0:
            count += 1
        i += 1
    return count


def FN(theta, pred, result):
    count = 0
    i = 0
    for x in (result):
        if pred[i][1] < theta and x == 1:
            count += 1
        i += 1
    return count


class ROC_curves:
    def __init__(self, pred, res, gran = 101):
        x = []
        y = []
        for j in range(0,gran):
            tn = TN(j/(gran-1),pred,res)
            neg = tn + FP(j/(gran-1),pred,res)
            tp = TP(j/(gran-1),pred,res)
            pos = tp + FN(j/(gran-1),pred,res)

            if neg == 0:
                x.append(0)
            else:
                x.append(tn / neg)
            if pos == 0:
                y.append(0)
            else:
                y.append(tp / pos)
      
        self.rocx = x
        self.rocy = y
    def plot_it(self):
        plt.plot(self.rocx,self.rocy)
        plt.show()

In [88]:
def algo_apprentissage(mod, data, target, print_mode=True, cv=10):
    
    X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=42)
    result = dict()
    mod_fit = mod.fit(X_train, y_train)
    y_prediction_proba = mod_fit.predict_proba(X_test)
    y_prediction = mod_fit.predict(X_test)
    q_mod = Qualite(_modele=mod,
                 _data=X_test,
                 _y=y_test,
                 _y_pred=y_prediction,
                 _y_pred_proba=y_prediction_proba,
                 _print_mode=print_mode,
                 _cv=cv)
    
    result['y_prediction_proba'] = y_prediction_proba
    result['y_prediction'] = y_prediction
    result['q_mod'] = q_mod
    
    if print_mode:
        print('La précision est ' + str(q_mod.accuracy))
        print('')
        print('L\'AUC est ' + str(q_mod.AUC))
        print('')
        print('Le score moyen par ' + str(cv) + ' cross-validation est ' + str(q_mod.moyenne))
    
    return result

# Parametres XGOOST

In [89]:
def param_XGB(data,
              target,
              max_depths=[10,50,100],
              learning_rates=[0.1,0.01],
              n_estimatorss=[50,100],
              sound_alarm=True):
    gamma=0.5
    reg_alpha=0.1
    colonnes = ['max_depth', 'learning_rates', 'n_estimators', 'score_moyen', 'qualite']
    result = pd.DataFrame(columns=colonnes)
    
    for md in max_depths:
        for lr in learning_rates:
            for ne in n_estimatorss:
                GBM = xgb.XGBClassifier(max_depth = md, 
                        n_estimators = ne, 
                        learning_rate = lr,
                        gamma = gamma,
                        reg_alpha= reg_alpha)
                apprentissage = algo_apprentissage(mod=GBM, data=data, target=target, print_mode=False)
                event = pd.DataFrame([[md, lr, ne, apprentissage['q_mod'].moyenne, apprentissage['q_mod']]], columns=colonnes)
                result = result.append(event)
                
    if sound_alarm:
        os.system('say "done"')
        
    result = result.reset_index()
    return result

# Attention, cette fonction mets du temps
#table_param = param_XGB(data=X, target=target)

# Transformation PCA

In [90]:
def pca_transform(data, target, variance=0.7, print_mode=True, normalized=True, min_max_ed=False):
    if normalized:
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
    elif min_max_ed:
        scaler = MinMaxScaler()
        data = scaler.fit_transform(data)
        
        
    for x in range(len(data[0])):
        pca = PCA(n_components=x)
        pca.fit(data)
        if sum(pca.explained_variance_ratio_) > variance:## and x >1:
            if print_mode:
                print('Hello World')
            break
    X_pca = pca.fit_transform(data)
    if print_mode:
        plt.scatter(X_pca[:, 0], X_pca[:, 1])
        plt.show()
        plt.close()
    return [pca, X_pca]
        
        
#pca_data = pca_transform(data=data['cleaned_data'], target=data['target'], normalized=True, print_mode=False)




In [91]:
def plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cvalidation=None,
                        n_jobs=4, train_sizes=np.linspace(.1, 1.0, 5), print_mode=True):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cvalidation : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cvalidation, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    if print_mode:
        plt.show()
        plt.close()
    return plt


title = "Learning Curves"
cvalidation = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = KNN
# plot_learning_curve(estimator, title, X=data['cleaned_data'], y=data['target'], ylim=(0.4, 1.01), cvalidation=cvalidation, n_jobs=4)




In [92]:
def final_table(data,
                target,
                col=colnames,
                algos=algos,
                indexs=indexs,
                mode=None,
               print_mode=True):
    if mode is None:
        pass
    elif mode == 'normalized':
        scaler = StandardScaler()
        scaler.fit(data)
        data = scaler.transform(data)
    elif mode == 'min_max_ed':
        scaler = MinMaxScaler()
        scaler.fit(data)
        data = scaler.transform(data)
    elif mode == 'pca':
        data = pca_transform(data=data,
                             target=target,
                                 normalized=True,
                                 print_mode=False)[1]


    result = []
    for alg in algos:
        t1 = time.time()
        res_algo = algo_apprentissage(alg, data=data, target=target, print_mode=print_mode)
        t2 = time.time()
        delta_t = t2 - t1
        ajout = [res_algo['q_mod'].accuracy, 
                 res_algo['q_mod'].AUC, 
                 res_algo['q_mod'].moyenne, 
                 res_algo['q_mod'].ecart_type, 
                 delta_t]
        result.append(ajout)
        if print_mode:
            cvalidation = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
            plot_learning_curve(alg, 'str(alg)', X=data, y=target, ylim=(0.4, 1.01), cvalidation=cvalidation, n_jobs=4)



    result = pd.DataFrame(result, columns=colnames, index=indexs)
    return result



In [93]:
def comparisons_tables(data, target, modes=modes, print_mode=True):
    tables = dict()
    for mode in modes:
        event = dict()
        tables[str(mode)] = final_table(data=data, target=target, mode=mode, print_mode=print_mode)
    
    return tables

In [96]:
# Attention, cela prend beaucoup de temps
final = final_table(X, target, print_mode=False)

/Users/ppx/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A colum

/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [98]:
final


,Accuracy,AUC,Moyenne CV,Std CV,Temps
KNN,0.924664,0.718121,0.881618,0.010405,193.837473
DTC,0.972197,0.927192,0.961470,0.016905,41.165508
id3,0.971300,0.926675,0.960569,0.012082,25.193742
BC,0.977578,0.941650,0.963264,0.016145,186.386689
RFC,0.973991,0.902685,0.937242,0.013775,6.980358
ABC,0.971300,0.906808,0.960512,0.013521,122.062243
DS,0.919283,0.820025,0.925603,0.019530,204.771032
GBM,0.973991,0.916875,0.960593,0.017435,245.354179


In [146]:
transformer = TfidfTransformer(smooth_idf=False)

In [147]:
tfidf = transformer.fit_transform(X)

In [148]:
tfidf.toarray

<bound method _cs_matrix.toarray of <5572x500 sparse matrix of type '<class 'numpy.float64'>'
	with 50729 stored elements in Compressed Sparse Row format>>

In [151]:
transformer = TfidfTransformer()
tfidf = pd.DataFrame(transformer.fit_transform(X).toarray())

In [153]:
final_tfidf = final_table(tfidf, target, print_mode=False)

/Users/ppx/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A colum

/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ppx/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [154]:
final_tfidf

,Accuracy,AUC,Moyenne CV,Std CV,Temps
KNN,0.942601,0.788073,0.880693,0.010152,9.912324
DTC,0.968610,0.916608,0.945294,0.017166,1.495918
id3,0.968610,0.933636,0.947956,0.014975,0.983708
BC,0.978475,0.947844,0.953337,0.012642,23.437925
RFC,0.975785,0.926425,0.965942,0.008695,0.587655
ABC,0.976682,0.924104,0.961494,0.014899,3.798798
DS,0.938117,0.822380,0.933606,0.023811,5.521707
GBM,0.977578,0.921784,0.952501,0.016442,13.944350
